In [16]:
# import dependencies
import pandas as pd
import sqlite3
from datetime import datetime, timedelta


In [17]:
# Create your connections with indoor and outdoor databases
Ocnx = sqlite3.connect('Outdoor.db')
Icnx = sqlite3.connect('Indoor.db')
Outdoor_df = pd.read_sql_query("SELECT * FROM BME_DATA", Ocnx)
Indoor_df = pd.read_sql_query("SELECT * FROM BME_DATA", Icnx)

In [18]:
# Convert time into datetime
Outdoor_df['TIME_STAMP'] = pd.to_datetime(Outdoor_df['TIME_STAMP'])
Indoor_df['TIME_STAMP'] = pd.to_datetime(Indoor_df['TIME_STAMP'])

In [19]:
Outdoor_df['TIME_STAMP'] = Outdoor_df['TIME_STAMP'].dt.round('1min')
Indoor_df['TIME_STAMP'] = Indoor_df['TIME_STAMP'].dt.round('1min')

In [20]:
Outdoor_df.head()

,id,TIME_STAMP,TEMPERATURE,GAS,HUMIDITY,PRESSURE,ALTITUDE
0,1,2019-05-12 18:46:00,75.0,100,50.0,1000.0,1000.0
1,2,2019-05-12 18:58:00,23.8,364884,84.9,975.0,323.2
2,3,2019-05-12 18:59:00,23.8,362475,84.6,975.0,323.3
3,4,2019-05-12 19:00:00,23.8,362874,84.2,975.0,323.4
4,5,2019-05-12 19:01:00,23.8,362475,84.4,975.0,323.5


In [21]:
# Deleting the wrong input at the first rows
Outdoor_df = Outdoor_df.iloc[1:]
Indoor_df = Indoor_df.iloc[1:]

In [22]:
# converting temperature from C to f
Outdoor_df['TEMPERATURE'] = round((Outdoor_df['TEMPERATURE']* 9/5) + 32)
Indoor_df['TEMPERATURE'] = round((Indoor_df['TEMPERATURE']* 9/5) + 32)

In [23]:
Outdoor_df.head()

,id,TIME_STAMP,TEMPERATURE,GAS,HUMIDITY,PRESSURE,ALTITUDE
1,2,2019-05-12 18:58:00,75.0,364884,84.9,975.0,323.2
2,3,2019-05-12 18:59:00,75.0,362475,84.6,975.0,323.3
3,4,2019-05-12 19:00:00,75.0,362874,84.2,975.0,323.4
4,5,2019-05-12 19:01:00,75.0,362475,84.4,975.0,323.5
5,6,2019-05-12 19:02:00,75.0,362874,84.4,975.0,323.5


In [24]:
Indoor_df.head()

,id,TIME_STAMP,TEMPERATURE,GAS,HUMIDITY,PRESSURE,ALTITUDE
1,2,2019-05-12 18:58:00,74.0,6049661,35.9,972.9,341.2
2,3,2019-05-12 18:59:00,74.0,5851528,35.9,972.9,341.3
3,4,2019-05-12 19:00:00,74.0,5636172,36.2,972.9,341.4
4,5,2019-05-12 19:01:00,74.0,5441624,37.3,972.9,341.5
5,6,2019-05-12 19:02:00,74.0,5219085,37.1,972.9,341.6


In [25]:
# Merging Indor and Outdoor (on time stamp)
master_df = pd.merge(Outdoor_df, Indoor_df, on = "TIME_STAMP", how = "left", suffixes=("_Out","_In"))


In [ ]:
# master_df = master_df.loc[:,["TIME_STAMP", "TEMPERATURE_Out", "GAS_Out", "PRESSURE_Out", "TEMPERATURE_In"]]

In [26]:
master_df.head()

,id_Out,TIME_STAMP,TEMPERATURE_Out,GAS_Out,HUMIDITY_Out,PRESSURE_Out,ALTITUDE_Out,id_In,TEMPERATURE_In,GAS_In,HUMIDITY_In,PRESSURE_In,ALTITUDE_In
0,2,2019-05-12 18:58:00,75.0,364884,84.9,975.0,323.2,2.0,74.0,6049661.0,35.9,972.9,341.2
1,3,2019-05-12 18:59:00,75.0,362475,84.6,975.0,323.3,3.0,74.0,5851528.0,35.9,972.9,341.3
2,4,2019-05-12 19:00:00,75.0,362874,84.2,975.0,323.4,4.0,74.0,5636172.0,36.2,972.9,341.4
3,5,2019-05-12 19:01:00,75.0,362475,84.4,975.0,323.5,5.0,74.0,5441624.0,37.3,972.9,341.5
4,6,2019-05-12 19:02:00,75.0,362874,84.4,975.0,323.5,6.0,74.0,5219085.0,37.1,972.9,341.6


In [27]:
# Convert datetime into float
#master_df['TIME_STAMP'] = pd.to_numeric(master_df['TIME_STAMP'], downcast = 'float')

In [29]:
# Export to csv
master_df.to_csv("Merged_Data.csv", sep=',', encoding='utf-8')